In [2]:
import numpy as np                          # Basic Array and Numaric Operation
import os                                   # use to access the files 
import tarfile                              # use to extract dataset from zip files
import sys
import zipfile                              # to extract zip file


import torch                                # Provides basic tensor operation and nn operation
import torchvision                          # Provides facilities to access image dataset


import my_utils.loadDataset as dl           # create dataloader for selected dataset
import my_utils.loadModel as lm             # facilitate loading and manipulating models
import my_utils.trainModel as tm            # Facilitate training of the model
import my_utils.initialize_pruning as ip    # Initialize and provide basic parmeter require for pruning
import my_utils.facilitate_pruning as fp    # Compute Pruning Value and many things

## Data Loader

In [3]:
# set the locationn of the dataset and trai and test data folder name
dl.setFolderLocation(datasets       ='/home/pragnesh/Dataset/',
                     selectedDataset='IntelIC/',
                     train          ='train',
                     test           ='test')
# set the imge properties
dl.setImageSize(224)
dl.setBatchSize = 2
dataLoader = dl.dataLoader()

#load the saved model if have any
load_path = "/home/pragnesh/Model/VGG_IntelIC_v2"
device1 = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
newModel = torch.load(load_path, map_location=torch.device('cpu'))

#if dont have any saved trained model download pretrained model for tranfer learning
# newmodel = lm.load_model(model_name='vgg16',number_of_class=6,pretrainval=False,
#                         freeze_feature=False,device_l=device1)

#print(newModel)

## Trainer

In [ ]:
logFile = '/home/pragnesh/Dataset/Intel_Image_Classifacation_v2/Logs/ConvModelv2.log'
#tm.device = torch.device('cpu')
tm.fit_one_cycle(#set locations of the dataset, train and test data
                 dataloaders=dataLoader,trainDir=dl.trainDir,testDir=dl.testDir,
                 # Selecat a variant of VGGNet
                 ModelName='vgg16',model=newModel,device_l=device1,
                 # Set all the Hyper-Parameter for training
                 epochs=1, max_lr=0.01, weight_decay=0, L1=0, grad_clip=.1, logFile=logFile)

#Save the  trained model 
SavePath = '/home/pragnesh/Model/vgg16-v2'
torch.save(newModel, SavePath)

## Pruning

#### Pruning Initialization

In [4]:
blockList   = ip.createBlockList(newModel)              #ip.getBlockList('vgg16')
featureList = ip.createFeatureList(newModel)
convIdx     = ip.findConvIndex(newModel)
module      = ip.getPruneModule(newModel)
prune_count = ip.getPruneCount(module=module,blocks=blockList,maxpr=.1)
print(f"Block List   = {blockList}\n"
      f"Feature List = {featureList}\n" 
      f"Conv Index   = {convIdx}\n"
      f"Prune Count  = {prune_count}"
      
     )
# for i in range(len(module)):
#     print(f"{i+1} :{module[i]}")

Block List   = [2, 2, 3, 3, 3]
Feature List = [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
Conv Index   = [0, 2, 5, 7, 10, 12, 14, 17, 19, 21, 24, 26, 28]
Prune Count  = [1, 1, 3, 3, 9, 9, 9, 26, 26, 26, 51, 51, 51]


#### Implementing custom pruning process

In [5]:
import torch.nn.utils.prune as prune
class KernalPruningMethod(prune.BasePruningMethod):
    PRUNING_TYPE = 'unstructured'
    def compute_mask(self, t, default_mask):
        mask = default_mask.clone()
        #mask.view(-1)[::2] = 0
        size = t.shape
        print(size)
        for k1 in range(len(newList[layer_number-st])):
            for k2 in range(len(newList[layer_number-st][k1])):
                i= newList[layer_number-st][k1][k2][1]
                j= newList[layer_number-st][k1][k2][0]
                
                #print(f"i= {i} , j= {j}")
                
                mask[i][j] = 0
        return mask
def kernal_unstructured(module, name):
    KernalPruningMethod.apply(module, name)
    return module

#### After pruning create new model with updated pruning list

In [6]:
def updateFeatureList(featureList,prune_count,start=0,end=len(prune_count)):
    j=0
    i=st
    while j < en:
        if featureList[i] == 'M':
            i+=1
            continue
        else:
            featureList[i] = featureList[i] - prune_count[j]
            j+=1
            i+=1
    return featureList


#### Copy the non zero weight value from prune model to new model 

In [30]:
def deepCopy(destModel,sourceModel,featureList):
    count=0
    for i in range(len(sourceModel.features)):
        if str(sourceModel.features[i]).find('Conv') != -1:
            size = sourceModel.features[i]._parameters['weight'].shape
            print(size)
            for fin in range(size[1]):
                j=0
                for fout in range(size[0]):
                    if torch.norm(sourceModel.features[i]._parameters['weight'][fout][fin]) != 0:
                        #print(f'i ={i} , j={j}, fin = {fin}, fout= {fout}')
                        if j>=featureList[count]:
                            break
                        print("i= ",i," j=",j," featureList[count]=",featureList[count])
                        
                        t = sourceModel.features[i]._parameters['weight'][fout][fin]
                        destModel.features[i]._parameters['weight'][j][fin]=t
                        
                        j = j+1
            count = count +1


### Perform Pruning Blockwise For Each Layer of Block

In [10]:
st=0
en=0
layer_number = 0
candidateConvLayer = []
newList = []
for bl in range(len(blockList)):
    if bl==1:
        break
    
    if bl==0:
        st = 0
    else:
        st=en
    en = en+blockList[bl]
    
    
    newList = []
    for i in range(st,en):
        layer_number =i
        
        candidateConvLayer.append(fp.compute_distance_score(module[i]._parameters['weight'],threshold=1))
        fp.sort_kernel_by_distance(candidateConvLayer[i])
        newList.append( fp.get_k_element(channel_list=candidateConvLayer[i],k=prune_count[st+i]) )
        
        kernal_unstructured(module=module[i],name='weight')
        prune.remove(module[i], 'weight')
        
        ###############IncompleteCode############
        
print("\n\n\nHere is the :",len(candidateConvLayer))


Shape of the tensor: torch.Size([64, 3, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 3])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 3
...torch.Size([64, 3, 3, 3])

Shape of the tensor: torch.Size([64, 64, 3, 3])
Print the Dims we want to keep: [0, 1]
norm shape = torch.Size([64, 64])
Number Of Features Map in current  layer l     = 64
Number Of Features Map in previous layer (l-1) = 64
................................................................torch.Size([64, 64, 3, 3])



Here is the : 2


### Create new temp model with updated feature list


In [12]:
featureList = updateFeatureList(featureList=featureList,prune_count=prune_count,start=0,end=2)
print(featureList)

[63, 63, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']


In [16]:
tempModel = lm.create_vgg_from_feature_list(featureList)
lm.freeze(tempModel,'vgg16')
print(tempModel)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 63, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(63, 63, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(63, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [31]:
deepCopy(destModel=tempModel,sourceModel=newModel,featureList=featureList)

torch.Size([64, 3, 3, 3])
i=  0  j= 0  featureList[count]= 63
i=  0  j= 1  featureList[count]= 63
i=  0  j= 2  featureList[count]= 63
i=  0  j= 3  featureList[count]= 63
i=  0  j= 4  featureList[count]= 63
i=  0  j= 5  featureList[count]= 63
i=  0  j= 6  featureList[count]= 63
i=  0  j= 7  featureList[count]= 63
i=  0  j= 8  featureList[count]= 63
i=  0  j= 9  featureList[count]= 63
i=  0  j= 10  featureList[count]= 63
i=  0  j= 11  featureList[count]= 63
i=  0  j= 12  featureList[count]= 63
i=  0  j= 13  featureList[count]= 63
i=  0  j= 14  featureList[count]= 63
i=  0  j= 15  featureList[count]= 63
i=  0  j= 16  featureList[count]= 63
i=  0  j= 17  featureList[count]= 63
i=  0  j= 18  featureList[count]= 63
i=  0  j= 19  featureList[count]= 63
i=  0  j= 20  featureList[count]= 63
i=  0  j= 21  featureList[count]= 63
i=  0  j= 22  featureList[count]= 63
i=  0  j= 23  featureList[count]= 63
i=  0  j= 24  featureList[count]= 63
i=  0  j= 25  featureList[count]= 63
i=  0  j= 26  featureL

i=  2  j= 35  featureList[count]= 63
i=  2  j= 36  featureList[count]= 63
i=  2  j= 37  featureList[count]= 63
i=  2  j= 38  featureList[count]= 63
i=  2  j= 39  featureList[count]= 63
i=  2  j= 40  featureList[count]= 63
i=  2  j= 41  featureList[count]= 63
i=  2  j= 42  featureList[count]= 63
i=  2  j= 43  featureList[count]= 63
i=  2  j= 44  featureList[count]= 63
i=  2  j= 45  featureList[count]= 63
i=  2  j= 46  featureList[count]= 63
i=  2  j= 47  featureList[count]= 63
i=  2  j= 48  featureList[count]= 63
i=  2  j= 49  featureList[count]= 63
i=  2  j= 50  featureList[count]= 63
i=  2  j= 51  featureList[count]= 63
i=  2  j= 52  featureList[count]= 63
i=  2  j= 53  featureList[count]= 63
i=  2  j= 54  featureList[count]= 63
i=  2  j= 55  featureList[count]= 63
i=  2  j= 56  featureList[count]= 63
i=  2  j= 57  featureList[count]= 63
i=  2  j= 58  featureList[count]= 63
i=  2  j= 59  featureList[count]= 63
i=  2  j= 60  featureList[count]= 63
i=  2  j= 61  featureList[count]= 63
i

i=  2  j= 22  featureList[count]= 63
i=  2  j= 23  featureList[count]= 63
i=  2  j= 24  featureList[count]= 63
i=  2  j= 25  featureList[count]= 63
i=  2  j= 26  featureList[count]= 63
i=  2  j= 27  featureList[count]= 63
i=  2  j= 28  featureList[count]= 63
i=  2  j= 29  featureList[count]= 63
i=  2  j= 30  featureList[count]= 63
i=  2  j= 31  featureList[count]= 63
i=  2  j= 32  featureList[count]= 63
i=  2  j= 33  featureList[count]= 63
i=  2  j= 34  featureList[count]= 63
i=  2  j= 35  featureList[count]= 63
i=  2  j= 36  featureList[count]= 63
i=  2  j= 37  featureList[count]= 63
i=  2  j= 38  featureList[count]= 63
i=  2  j= 39  featureList[count]= 63
i=  2  j= 40  featureList[count]= 63
i=  2  j= 41  featureList[count]= 63
i=  2  j= 42  featureList[count]= 63
i=  2  j= 43  featureList[count]= 63
i=  2  j= 44  featureList[count]= 63
i=  2  j= 45  featureList[count]= 63
i=  2  j= 46  featureList[count]= 63
i=  2  j= 47  featureList[count]= 63
i=  2  j= 48  featureList[count]= 63
i

i=  2  j= 24  featureList[count]= 63
i=  2  j= 25  featureList[count]= 63
i=  2  j= 26  featureList[count]= 63
i=  2  j= 27  featureList[count]= 63
i=  2  j= 28  featureList[count]= 63
i=  2  j= 29  featureList[count]= 63
i=  2  j= 30  featureList[count]= 63
i=  2  j= 31  featureList[count]= 63
i=  2  j= 32  featureList[count]= 63
i=  2  j= 33  featureList[count]= 63
i=  2  j= 34  featureList[count]= 63
i=  2  j= 35  featureList[count]= 63
i=  2  j= 36  featureList[count]= 63
i=  2  j= 37  featureList[count]= 63
i=  2  j= 38  featureList[count]= 63
i=  2  j= 39  featureList[count]= 63
i=  2  j= 40  featureList[count]= 63
i=  2  j= 41  featureList[count]= 63
i=  2  j= 42  featureList[count]= 63
i=  2  j= 43  featureList[count]= 63
i=  2  j= 44  featureList[count]= 63
i=  2  j= 45  featureList[count]= 63
i=  2  j= 46  featureList[count]= 63
i=  2  j= 47  featureList[count]= 63
i=  2  j= 48  featureList[count]= 63
i=  2  j= 49  featureList[count]= 63
i=  2  j= 50  featureList[count]= 63
i

i=  2  j= 25  featureList[count]= 63
i=  2  j= 26  featureList[count]= 63
i=  2  j= 27  featureList[count]= 63
i=  2  j= 28  featureList[count]= 63
i=  2  j= 29  featureList[count]= 63
i=  2  j= 30  featureList[count]= 63
i=  2  j= 31  featureList[count]= 63
i=  2  j= 32  featureList[count]= 63
i=  2  j= 33  featureList[count]= 63
i=  2  j= 34  featureList[count]= 63
i=  2  j= 35  featureList[count]= 63
i=  2  j= 36  featureList[count]= 63
i=  2  j= 37  featureList[count]= 63
i=  2  j= 38  featureList[count]= 63
i=  2  j= 39  featureList[count]= 63
i=  2  j= 40  featureList[count]= 63
i=  2  j= 41  featureList[count]= 63
i=  2  j= 42  featureList[count]= 63
i=  2  j= 43  featureList[count]= 63
i=  2  j= 44  featureList[count]= 63
i=  2  j= 45  featureList[count]= 63
i=  2  j= 46  featureList[count]= 63
i=  2  j= 47  featureList[count]= 63
i=  2  j= 48  featureList[count]= 63
i=  2  j= 49  featureList[count]= 63
i=  2  j= 50  featureList[count]= 63
i=  2  j= 51  featureList[count]= 63
i

i=  2  j= 44  featureList[count]= 63
i=  2  j= 45  featureList[count]= 63
i=  2  j= 46  featureList[count]= 63
i=  2  j= 47  featureList[count]= 63
i=  2  j= 48  featureList[count]= 63
i=  2  j= 49  featureList[count]= 63
i=  2  j= 50  featureList[count]= 63
i=  2  j= 51  featureList[count]= 63
i=  2  j= 52  featureList[count]= 63
i=  2  j= 53  featureList[count]= 63
i=  2  j= 54  featureList[count]= 63
i=  2  j= 55  featureList[count]= 63
i=  2  j= 56  featureList[count]= 63
i=  2  j= 57  featureList[count]= 63
i=  2  j= 58  featureList[count]= 63
i=  2  j= 59  featureList[count]= 63
i=  2  j= 60  featureList[count]= 63
i=  2  j= 61  featureList[count]= 63
i=  2  j= 62  featureList[count]= 63
i=  2  j= 0  featureList[count]= 63
i=  2  j= 1  featureList[count]= 63
i=  2  j= 2  featureList[count]= 63
i=  2  j= 3  featureList[count]= 63
i=  2  j= 4  featureList[count]= 63
i=  2  j= 5  featureList[count]= 63
i=  2  j= 6  featureList[count]= 63
i=  2  j= 7  featureList[count]= 63
i=  2  j=

i=  2  j= 49  featureList[count]= 63
i=  2  j= 50  featureList[count]= 63
i=  2  j= 51  featureList[count]= 63
i=  2  j= 52  featureList[count]= 63
i=  2  j= 53  featureList[count]= 63
i=  2  j= 54  featureList[count]= 63
i=  2  j= 55  featureList[count]= 63
i=  2  j= 56  featureList[count]= 63
i=  2  j= 57  featureList[count]= 63
i=  2  j= 58  featureList[count]= 63
i=  2  j= 59  featureList[count]= 63
i=  2  j= 60  featureList[count]= 63
i=  2  j= 61  featureList[count]= 63
i=  2  j= 62  featureList[count]= 63
i=  2  j= 0  featureList[count]= 63
i=  2  j= 1  featureList[count]= 63
i=  2  j= 2  featureList[count]= 63
i=  2  j= 3  featureList[count]= 63
i=  2  j= 4  featureList[count]= 63
i=  2  j= 5  featureList[count]= 63
i=  2  j= 6  featureList[count]= 63
i=  2  j= 7  featureList[count]= 63
i=  2  j= 8  featureList[count]= 63
i=  2  j= 9  featureList[count]= 63
i=  2  j= 10  featureList[count]= 63
i=  2  j= 11  featureList[count]= 63
i=  2  j= 12  featureList[count]= 63
i=  2  j= 1

i=  2  j= 3  featureList[count]= 63
i=  2  j= 4  featureList[count]= 63
i=  2  j= 5  featureList[count]= 63
i=  2  j= 6  featureList[count]= 63
i=  2  j= 7  featureList[count]= 63
i=  2  j= 8  featureList[count]= 63
i=  2  j= 9  featureList[count]= 63
i=  2  j= 10  featureList[count]= 63
i=  2  j= 11  featureList[count]= 63
i=  2  j= 12  featureList[count]= 63
i=  2  j= 13  featureList[count]= 63
i=  2  j= 14  featureList[count]= 63
i=  2  j= 15  featureList[count]= 63
i=  2  j= 16  featureList[count]= 63
i=  2  j= 17  featureList[count]= 63
i=  2  j= 18  featureList[count]= 63
i=  2  j= 19  featureList[count]= 63
i=  2  j= 20  featureList[count]= 63
i=  2  j= 21  featureList[count]= 63
i=  2  j= 22  featureList[count]= 63
i=  2  j= 23  featureList[count]= 63
i=  2  j= 24  featureList[count]= 63
i=  2  j= 25  featureList[count]= 63
i=  2  j= 26  featureList[count]= 63
i=  2  j= 27  featureList[count]= 63
i=  2  j= 28  featureList[count]= 63
i=  2  j= 29  featureList[count]= 63
i=  2  j

i=  2  j= 16  featureList[count]= 63
i=  2  j= 17  featureList[count]= 63
i=  2  j= 18  featureList[count]= 63
i=  2  j= 19  featureList[count]= 63
i=  2  j= 20  featureList[count]= 63
i=  2  j= 21  featureList[count]= 63
i=  2  j= 22  featureList[count]= 63
i=  2  j= 23  featureList[count]= 63
i=  2  j= 24  featureList[count]= 63
i=  2  j= 25  featureList[count]= 63
i=  2  j= 26  featureList[count]= 63
i=  2  j= 27  featureList[count]= 63
i=  2  j= 28  featureList[count]= 63
i=  2  j= 29  featureList[count]= 63
i=  2  j= 30  featureList[count]= 63
i=  2  j= 31  featureList[count]= 63
i=  2  j= 32  featureList[count]= 63
i=  2  j= 33  featureList[count]= 63
i=  2  j= 34  featureList[count]= 63
i=  2  j= 35  featureList[count]= 63
i=  2  j= 36  featureList[count]= 63
i=  2  j= 37  featureList[count]= 63
i=  2  j= 38  featureList[count]= 63
i=  2  j= 39  featureList[count]= 63
i=  2  j= 40  featureList[count]= 63
i=  2  j= 41  featureList[count]= 63
i=  2  j= 42  featureList[count]= 63
i

IndexError: index 63 is out of bounds for dimension 0 with size 63